In [416]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc

In [376]:
df = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/NCCR_combined_corpus_DE_wording_available_TEST.csv")

In [394]:
nlp = spacy.load("de_core_news_lg", exclude=['tok2vec', 'tagger', 'morphologizer', 'parser',
                         'attribute_ruler', 'lemmatizer'])

nlp.add_pipe("sentencizer")



In [395]:
def standardize_text(text):
    # Replace special characters #todo: simplify replacement
    text = text.replace("ä", "ae").replace("ü", "ue").replace("ö", "oe").replace("Ö", "Oe").replace("Ä", "Ae").replace("Ü", "Ue").replace("ß", "ss").replace("@", "").replace("/", "")
    text = text.replace("ç", "c"). replace("", "")
    
    text = text.lower()
    text = " ".join(text.split())

    return text

In [396]:
def standardize_wording(wording):
    # Replace special characters #todo: simplify replacement
    wording = wording.replace("/", "").replace("@", "").replace("<ord:65430>", "")
    wording = " ".join(wording.split())

    return wording

In [397]:
df['text_prep'] = df['text_prep'].apply(lambda x: standardize_text(x))

In [398]:
df['Wording'] = df['Wording'].apply(lambda x: standardize_wording(x))

In [400]:
df['text_prep'] 

0      "bucher fordert ehrenerklaerung von allen parl...
1      na berlin (ots) - der rheinland-pfaelzische fi...
2      wallner: gruene transparenz-offensive soll kla...
3      zum geheimdienstskandal weltweit umfassender a...
4      "genug gezahlt - ueberstunden steuerfrei", "jo...
                             ...                        
944    na berlin (ots) - buendnis 90die gruenen haben...
945    na berlin. vor der parlamentswahl in simbabwe ...
946    na muenchen die 600 plaetze der festhalle des ...
947    bilaterale ja eu-beitritt nein. dank den bilat...
948    na bemaengelte oekonomisierung von ehe und fam...
Name: text_prep, Length: 949, dtype: object

In [401]:
df['Wording'] 

0      bucher fordert ehrenerklaerung von allen parla...
1      sein parteikollege, der rheinland-pfaelzische ...
2      das team stronach hat bis april 2013 laut rech...
3                                                    wir
4      waehrend andere parteien mit nackter haut vers...
                             ...                        
944                                              gruenen
945    die fdp-bundestagsfraktion fordert alle staatl...
946                                      die altparteien
947                                                  svp
948                                        norbert bluem
Name: Wording, Length: 949, dtype: object

In [425]:
df['doc'] = list(nlp.pipe(df['text_prep']))
df['wording_doc'] = list(nlp.pipe(df['Wording']))




In [424]:
doc = nlp('Das ist ein Test der das Ganze prüft')
get_sub_wording(doc)

Das ist ein Test der 

In [436]:
def get_sub_wording(wording):
    doc2 = Doc(wording.vocab, words=[t.text for i, t in enumerate(wording) if i <3])
    return doc2 
    


In [437]:
df['wording_doc'] = df['wording_doc'].apply(lambda x: get_sub_wording(x))

In [438]:
df['wording_doc']

0          (bucher, fordert, ehrenerklaerung)
1                    (sein, parteikollege, ,)
2                       (das, team, stronach)
3                                       (wir)
4                (waehrend, andere, parteien)
                        ...                  
944                                 (gruenen)
945    (die, fdp-bundestagsfraktion, fordert)
946                        (die, altparteien)
947                                     (svp)
948                          (norbert, bluem)
Name: wording_doc, Length: 949, dtype: object

In [439]:
# Define function to find index of tokens that match Wording content
def get_matches(doc, wording):
    
    # Define Spacy Matcher
    matcher = PhraseMatcher(nlp.vocab)
    # Add patterns from nlp-preprocessed Wording column
    matcher.add("WORDING", [wording])
    
    # Get matches
    matches = matcher(doc)

    return matches



In [440]:
df['matches_new'] = df.apply(lambda x: get_matches(x['doc'], x['wording_doc']), axis=1)

## Fix non-matches

In [441]:
df['matches_new']

0                          [(1212883764967918647, 1, 4)]
1                      [(1212883764967918647, 345, 348)]
2      [(1212883764967918647, 791, 794), (12128837649...
3      [(1212883764967918647, 29, 30), (1212883764967...
4                        [(1212883764967918647, 60, 63)]
                             ...                        
944    [(1212883764967918647, 8, 9), (121288376496791...
945                      [(1212883764967918647, 68, 71)]
946                    [(1212883764967918647, 117, 119)]
947                      [(1212883764967918647, 97, 98)]
948                      [(1212883764967918647, 19, 21)]
Name: matches_new, Length: 949, dtype: object

In [442]:
non = df[~df['matches_new'].astype(bool)]
non = non[['ID', 'doc', 'Wording', 'matches_new', 'wording_segments']]
#non['wording_doc'] = list(nlp.pipe(non['Wording']))
non['doc_tokens'] = non['doc'].apply(lambda x: [token.text for token in x])
non['wording_tokens'] = non['wording_doc'].apply(lambda x: [token.text for token in x])

In [443]:
len(non)

47

In [444]:
non['Wording'].iloc[1]

'volksparte'

In [446]:
non['doc'].iloc[1]

1.6. oesterreich und europa. europaeische union. die volkspartei sieht in der zukuenftigen mitgliedschaft oesterreichs zur europaeischen union die wesentlichste voraussetzung zum ausbau unserer, wirtschaftskraft und zur sicherung der beschaeftigung in oesterreich. der zielstrebigkeit der volkspartei ist es zu verdanken, dass oesterreich am 17. juli 1989 den antrag auf beitritt zu den europaeischen gemeinschaften gestellt hat. am 12. juni 1994 haben die oesterreicherinnen und oesterreicher mit ueberwaeltigender mehrheit entschieden, dass oesterreich mit beginn des kommenden jahres der europaeischen union als gleichberechtigter partner angehoeren wird. damit wurde fuer oesterreich die wichtigste entscheidung seit dem abschluss des staatsvertrag zum wohle unserer heimat getroffen. die antwort der europaeischen union auf die weltweiten herausforderungen, vor allem im technischen und wirtschaftlichen bereich, sind der binnenmarkt und die einheitliche europaeische akte sowie deren weiterentw

In [447]:
print(str(non.iloc[2]['doc_tokens']))

['liberal-sozial', ':-)']


In [448]:
print(str(non.iloc[2]['wording_tokens']))

['barbara', 'schmid-federer']


In [ ]:
--> try match first 5 tokens

## Investigate Missing Segments

In [145]:
# sample type?
import pandas as pd
non_train= pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TRAIN.csv")
non_test = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TEST.csv")

In [146]:
df = non_test
df

,Unnamed: 0,level_0,index,ID,text,Coder,#TS,#TS2,Codierstart_Lab,Medium,...,year,text_temp,Wording_temp,doc_temp,Wording_doc_temp,wording_matches,wording_sentence,wording_segments,doc_tokens,wording_tokens
0,3830,3845,2936,de_pm_el_13_50003.txt,Text ID:\nde_pm_el_13_50003.txt\n\nParty Manif...,luan,1.409665e+09,1.409832e+09,Tue Sep 02 15:35:49 2014,1560,...,2013,ii . finanzkapitalismus baendigen  w irtschaf...,ii . finanzkapitalismus baendigen w irtschaft ...,ii . finanzkapitalismus baendigen  w irtschaf...,ii . finanzkapitalismus baendigen w,[],[],NaN,"['ii', '.', 'finanzkapitalismus', 'baendigen',...","['ii', '.', 'finanzkapitalismus', 'baendigen',..."
1,2779,2779,1459,cd_fb_db_13332_00039_2.txt,Text ID:\ncd_fb_db_13332_00039.txt\n\n\nFacebo...,lamo,1.470730e+09,1.470730e+09,Tue Aug 09 09:58:22 2016,9902,...,2015,nicht vergessen: am naechsten sonntag sind wah...,svp,nicht vergessen: am naechsten sonntag sind wah...,svp,[],[],NaN,"['nicht', 'vergessen', ':', 'am', 'naechsten',...",['svp']
2,428,428,375,au_pm_el_94_40009.txt,Text ID:\nau_pm_el_94_40009.txt\n\nParty Manif...,kasa,1.408028e+09,1.408030e+09,Thu Aug 14 16:48:25 2014,1160,...,1994,1.6. oesterreich und europa. europaeische unio...,volksparte,1.6. oesterreich und europa. europaeische unio...,volksparte,[],[],NaN,"['1.6.', 'oesterreich', 'und', 'europa', '.', ...",['volksparte']
3,2772,2772,1438,cd_fb_db_13296_00035z.txt,Text ID:\ncd_fb_db_13296_00035.txt\n\n\nFacebo...,lamo,1.470397e+09,1.470397e+09,Fri Aug 05 13:29:01 2016,9902,...,2015,liberal-sozial :-),barbara schmid-federer,liberal-sozial :-),barbara schmid-federer,[],[],NaN,"['liberal-sozial', ':-)']","['barbara', 'schmid-federer']"
4,2320,2325,1052,au_pr_el_13_060119.txt,Text-ID: at_pr_el_13_060119.txt\n\nPress Relea...,difr,1.434900e+09,1.434902e+09,Sun Jun 21 17:28:04 2015,1150,...,2013,60 prozent des einkommens mittlerweile in form...,"sterreichische steuerzahler hat mehr als ""genu...",60 prozent des einkommens mittlerweile in form...,sterreichische steuerzahler hat mehr als,[],[],NaN,"['60', 'prozent', 'des', 'einkommens', 'mittle...","['sterreichische', 'steuerzahler', 'hat', 'meh..."
5,6045,6045,3766,de_pr_el_13_090049.txt,Text-ID: de_pr_el_13_090049.txt\n\nPress Relea...,chfi,1.434038e+09,1.434039e+09,Thu Jun 11 17:46:27 2015,1550,...,2013,na diskussion um den soli zeigt die gier der p...,t der politik:,na diskussion um den soli zeigt die gier der p...,t der politik :,[],[],NaN,"['na', 'diskussion', 'um', 'den', 'soli', 'zei...","['t', 'der', 'politik', ':']"
6,4060,4060,3026,de_pm_el_94_20008.txt,Text ID:\nde_pm_el_94_20008.txt\n\nParty Manif...,pagr,1.404811e+09,1.404813e+09,Tue Jul 08 11:22:56 2014,1560,...,1994,"sichere energieversorgung, sparsamer energieve...",deshalb fordert die fdp.: - konkrete massnahme...,"sichere energieversorgung, sparsamer energieve...",deshalb fordert die fdp .,[],[],NaN,"['sichere', 'energieversorgung', ',', 'sparsam...","['deshalb', 'fordert', 'die', 'fdp', '.']"
7,1748,1748,860,au_pr_el_13_050045.txt,Text-ID: at_pr_el_13_050045.txt\n\nPress Relea...,stbu,1.416822e+09,1.416824e+09,Mon Nov 24 10:47:25 2014,1150,...,2013,ueberwaeltigende mehrheit der schweizer hat si...,fpoe: strache: die schweiz zeigt es vor - dire...,ueberwaeltigende mehrheit der schweizer hat si...,fpoe : strache : die,[],[],NaN,"['ueberwaeltigende', 'mehrheit', 'der', 'schwe...","['fpoe', ':', 'strache', ':', 'die']"
8,6123,6123,3946,de_tw_db_15038_00274z.txt,Text ID:\nde_tw_db_15038_00274.txt\n\n\nTweet ...,stbu,1.464358e+09,1.464360e+09,Fri May 27 16:10:06 2016,9901,...,2015,jetzt doch: die_gruenen wollen kaufpraemie fue...,gruenen wollen kaufpraemie fuer elektroautos h...,jetzt doch: die_gruenen wollen kaufpraemie fue...,gruenen wollen kaufpraemie fuer elektroautos,[],[],NaN,"['jetzt', 'doch', ':', 'die_gruenen', 'wollen'...","['gruenen', 'wollen', 'kaufpraemie', 'fuer', '..."
9,6187,6187,4108,de_tw_db_15058_01622.txt,Text ID:\nde_tw_db_15058_01622.txt\n\n\nTweet ...,mabu,1.457923e+09,1.457924e+0

In [135]:
list(non_train)

['Unnamed: 0',
 'level_0',
 'index',
 'ID',
 'text',
 'Coder',
 '#TS',
 '#TS2',
 'Codierstart_Lab',
 'Medium',
 'Author',
 'Author_ID',
 'Date',
 'Length',
 'TS_Beginning',
 'TS_End',
 'Images',
 'Position',
 'Genre',
 'Main_Issue',
 'Frame_HI_face',
 'Frame_HI_indi',
 'Frame_HI_emoN',
 'Frame_HI_emoP',
 'Frame_Con_disa',
 'Frame_Con_repr',
 'Frame_Con_winn',
 'Frame_Con_slant',
 'Frame_PRF_conduit',
 'Frame_PRF_context',
 'Frame_PRF_interp',
 'Frame_PRF_account',
 'Frame_Neg_tone',
 'Frame_Neg_scep',
 'Frame_Neg_other',
 'Bemerkungen',
 'Count_Speaker',
 'Count_Issues',
 'Count_ActEval',
 'STYLE_Negativ',
 'STYLE_Emot',
 'T_Brutto',
 'T_Break',
 'T_Netto',
 'T_H',
 'Sessiontext',
 'Session_ID',
 'Sample',
 'Sample_Country',
 'Sample_Lang',
 'Sample_Type',
 'Weight',
 'Filter_Coded',
 'STYLE_Colloquial',
 'STYLE_Casual',
 'STYLE_Facts',
 'STYLE_Sense',
 'STYLE_BlackWhite',
 'STYLE_Sarcasm',
 'STYLE_Drama',
 'STYLE_EmoTone',
 'STYLE_CommMan',
 'STYLE_UsThem',
 'STYLE_Privat',
 'STYLE_Co

In [136]:
df.iloc[3]['doc_tokens']

"['liberal-sozial', ':-)']"

In [137]:
df.iloc[3]['wording_tokens']
#non_train.iloc[221]['Wording']

"['barbara', 'schmid-federer']"

In [138]:
for index, row in df.iterrows():
    print(row['doc_tokens'])
    print(row['wording_tokens'])
    #print(row['Wording'])
    print("___")

['ii', '.', 'finanzkapitalismus', 'baendigen', '\x96', 'w', 'irtschaft', 'und', 'mittelstand', 'staerken', '\x96', 'g', 'ute', 'arbeit', 'schaffen', 'ii', '.1', 'm', 'aerkte', 'brauchen', 'regeln', '\x96', 'fuer', 'nachhaltiges', 'w', 'achstum', 'das', 'zeitalter', 'des', 'marktradikalismus', 'ist', 'zu', 'ende', '.', 'die', 'ideologie', 'von', 'konservativen', 'und', 'neoliberalen', ',', 'die', 'maerkte', 'sorgten', 'schon', 'selbst', 'fuer', 'das', 'gemeinwohl', 'und', 'die', 'steigerung', 'des', 'wohlstands', ',', 'ist', 'spaetestens', 'unter', 'den', 'truemmern', 'der', 'weltweiten', 'finanzkatastrophe', 'begraben', 'worden', '.', 'statt', 'nach', 'marktwirtschaftlichem', 'prinzip', 'risiko', 'und', 'haftung', 'in', 'einer', 'hand', 'zu', 'lassen', ',', 'hat', 'der', 'marktradikalismus', 'gewinne', 'privatisiert', 'und', 'verluste', 'der', 'allgemeinheit', 'aufgebuerdet', '.', 'er', 'hat', 'seine', 'eigenen', 'grundlagen', 'zerstoert', 'und', 'weltweit', 'billionen', 'von', 'schuld

### Evaluate how many additional double matches caused by only using n=5 first tokens


In [3]:
import pandas as pd
all_tokens_train =pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/token_analysis/NCCR_combined_corpus_DE_wording_available_TRAIN_all.csv")
n_tokens_train = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/token_analysis/NCCR_combined_corpus_DE_wording_available_TRAIN.csv")
all_tokens_test = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/token_analysis/NCCR_combined_corpus_DE_wording_available_TEST_all.csv")
n_tokens_test =  pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/token_analysis/NCCR_combined_corpus_DE_wording_available_TEST.csv")


In [87]:
def match_counter(x):
    res = x.split('), ')
    count = len(res)
    #print(len(res))
    #print(res)
    return count

def sentence_counter(x):
    res = x.split('], ')
    count = len(res)
    #print(len(res))
    #print(res)
    return count

n_tokens_train['match_count'] = n_tokens_train['wording_matches'].apply(lambda x: match_counter(x))
n_tokens_train['sent_count'] = n_tokens_train['wording_sentence'].apply(lambda x: sentence_counter(x))



pd.set_option('display.max_rows', None)
n_tokens_train.groupby('sent_count').count()



,Unnamed: 0,level_0,index,ID,text,Coder,#TS,#TS2,Codierstart_Lab,Medium,...,Wording,Fulltext,Source,text_prep,party,year,wording_matches,wording_sentence,wording_segments,match_count
sent_count,,,,,,,,,,,,,,,,,,,,,
1,3081,3081,3081,3081,3081,3081,3081,3081,3081,3081,...,3081,3037,3081,3081,2550,3081,3081,3081,2891,3081
2,275,275,275,275,275,275,275,275,275,275,...,275,274,275,275,262,275,275,275,275,275
3,155,155,155,155,155,155,155,155,155,155,...,155,155,155,155,132,155,155,155,155,155
4,81,81,81,81,81,81,81,81,81,81,...,81,81,81,81,76,81,81,81,81,81
5,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,34,41,41,41,41,41
6,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,29,33,33,33,33,33
7,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,15,20,20,20,20,20
8,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,7,8,8,8,8,8
9,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,13,18,18,18,18,18


In [70]:
x = n_tokens_train.iloc[0]['wording_sentence']
x


"[['na gitta schuessler (npd): akzeptanz des betreuungsgeldes in sachsen spricht fuer diesen ersten schritt auf dem weg zum muettergehalt beim sogenannten kanzler-duell am vergangenen sonntag bezeichnete der spd-kanzlerkandidat peer steinbrueck das vor einem monat eingefuehrte bundesbetreuungsgeld als saudaemlich.'], ['die npd plaediert seit langem fuer die einfuehrung eines muettergehalts, damit junge frauen wirklich frei waehlen koennen, ohne, wie jetzt, mit finanziellen einschnitten rechnen zu muessen.']]"

In [77]:
res = x.split('], ')
res


["[['na gitta schuessler (npd): akzeptanz des betreuungsgeldes in sachsen spricht fuer diesen ersten schritt auf dem weg zum muettergehalt beim sogenannten kanzler-duell am vergangenen sonntag bezeichnete der spd-kanzlerkandidat peer steinbrueck das vor einem monat eingefuehrte bundesbetreuungsgeld als saudaemlich.'",
 "['die npd plaediert seit langem fuer die einfuehrung eines muettergehalts, damit junge frauen wirklich frei waehlen koennen, ohne, wie jetzt, mit finanziellen einschnitten rechnen zu muessen.']]"]

In [4]:
all_tokens_test.loc[all_tokens_test.ID == 'au_pr_el_13_060087.txt']

,Unnamed: 0,level_0,index,ID,text,Coder,#TS,#TS2,Codierstart_Lab,Medium,...,Tgt_ID,Wording,Fulltext,Source,text_prep,party,year,wording_matches,wording_sentence,wording_segments
4,2232,2232,1026,au_pr_el_13_060087.txt,Text-ID: at_pr_el_13_060087.txt\n\nPress Relea...,andi,1.418247e+09,1.418251e+09,Wed Dec 10 22:36:40 2014,1150,...,SPty,Waehrend andere Parteien mit nackter Haut vers...,Im Zuge einer Pressekonferenz praesentierten B...,Target,"""Genug gezahlt - Überstunden steuerfrei"", ""Job...",bzo,2013,"[(1212883764967918647, 60, 124)]","[['""genug gezahlt - ueberstunden steuerfrei"", ...","""genug gezahlt - ueberstunden steuerfrei"", ""jo..."


In [5]:
s = all_tokens_test.loc[all_tokens_test.ID == 'au_pr_el_13_060087.txt']['text_prep']

In [6]:
all_tokens_test.loc[all_tokens_test.ID == 'au_pr_el_13_060087.txt']['Wording']

4    Waehrend andere Parteien mit nackter Haut vers...
Name: Wording, dtype: object

In [7]:
s

4    "Genug gezahlt - Überstunden steuerfrei", "Job...
Name: text_prep, dtype: object

In [8]:
all_tokens_test.loc[all_tokens_test.ID == 'au_pr_el_13_060087.txt']['text']

4    Text-ID: at_pr_el_13_060087.txt\n\nPress Relea...
Name: text, dtype: object

In [9]:
list(n_tokens_test)

['Unnamed: 0',
 'level_0',
 'index',
 'ID',
 'text',
 'Coder',
 '#TS',
 '#TS2',
 'Codierstart_Lab',
 'Medium',
 'Author',
 'Author_ID',
 'Date',
 'Length',
 'TS_Beginning',
 'TS_End',
 'Images',
 'Position',
 'Genre',
 'Main_Issue',
 'Frame_HI_face',
 'Frame_HI_indi',
 'Frame_HI_emoN',
 'Frame_HI_emoP',
 'Frame_Con_disa',
 'Frame_Con_repr',
 'Frame_Con_winn',
 'Frame_Con_slant',
 'Frame_PRF_conduit',
 'Frame_PRF_context',
 'Frame_PRF_interp',
 'Frame_PRF_account',
 'Frame_Neg_tone',
 'Frame_Neg_scep',
 'Frame_Neg_other',
 'Bemerkungen',
 'Count_Speaker',
 'Count_Issues',
 'Count_ActEval',
 'STYLE_Negativ',
 'STYLE_Emot',
 'T_Brutto',
 'T_Break',
 'T_Netto',
 'T_H',
 'Sessiontext',
 'Session_ID',
 'Sample',
 'Sample_Country',
 'Sample_Lang',
 'Sample_Type',
 'Weight',
 'Filter_Coded',
 'STYLE_Colloquial',
 'STYLE_Casual',
 'STYLE_Facts',
 'STYLE_Sense',
 'STYLE_BlackWhite',
 'STYLE_Sarcasm',
 'STYLE_Drama',
 'STYLE_EmoTone',
 'STYLE_CommMan',
 'STYLE_UsThem',
 'STYLE_Privat',
 'STYLE_Co

In [ ]:
'Mitgliedstaaten der EU'

In [20]:
all_tokens_test.loc[all_tokens_test.POPULIST == 1].iloc[10]['wording_sentence']
#all_tokens_test.loc[all_tokens_test.POPULIST == 1].iloc[10]['text']
#all_tokens_test.loc[all_tokens_test.POPULIST == 1].iloc[10]['Wording']



"[['die mitgliedstaaten der eu muessen den schutz ihrer minderheiten in rechtlicher und tatsaechlicher hinsicht gewaehrleisten.']]"